# работа Суворовой Марии, 204
(на всякий случай)

Первое, что делаем - импортируем всё, что требуется.

In [1]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split


In [2]:
morph = MorphAnalyzer()


Читаем файл train.csv:

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')


Создаём функцию, которая подготавливает данные: лемматизирует предложения, считает tf-idf и масштабирует.

In [13]:
def prepare_data_for_model(data):
    stops = stopwords.words("english")
    tfidf = TfidfVectorizer(
        analyzer="word",
        stop_words=stops
    )

    data_preprocessed = []
    for text in data:
        tokens = wordpunct_tokenize(text)
        text_lemmatized = " ".join(
            [morph.parse(item)[0].normal_form for item in tokens])
        data_preprocessed.append(text_lemmatized)

    data_tfidf = tfidf.fit_transform(data_preprocessed)
    # data_tfidf = pd.DataFrame(data_tfidf.toarray())
    # data_tfidf = data_tfidf[data_tfidf.columns[:76915]]
    data_tfidf = data_tfidf[0:25000, 0:76096]

    scaler = StandardScaler(with_mean=False)
    scaler.fit(data_tfidf)
    scaler.transform(data_tfidf)
    return data_tfidf


In [14]:
X_tr = prepare_data_for_model(train_df["text"].tolist())
y_tr = train_df["answer"].tolist()
X_te = prepare_data_for_model(test_df["text"].tolist())


Я выбрала метод KNN для данной задачи. Будут использоваться 5 соседей.

In [17]:
model5 = KNeighborsClassifier(n_neighbors=5)
model5.fit(X_tr, y_tr)
preds_te5 = model5.predict(X_te)


In [18]:
result5 = test_df[["id"]].copy()
result5["answer"] = preds_te5

result5.to_csv("result5.csv", index=False)
